In [52]:
import pandas as pd
from google.cloud import bigquery
from datetime import datetime, timedelta
import joblib
import numpy as np

In [53]:
# project_id = "banded-setting-428309-q4"
# dataset_id = "datos"

In [54]:
# client = bigquery.Client(project='banded-setting-428309-q4')
# one_week_ago = datetime.now() - timedelta(days=7)
    
# query = f"""
# SELECT
#     FORMAT_TIMESTAMP('%Y-%m-%d', Timestamp) AS Day,
#     FORMAT_TIMESTAMP('%H', Timestamp) AS Hour,
#     FORMAT_TIMESTAMP('%M', Timestamp) AS Minute,
#     ct.descripcion,
#     bd.Value
# FROM `banded-setting-428309-q4.datos.bronze-data` bd
# LEFT JOIN `banded-setting-428309-q4.datos.col-tag` ct on bd.Tag = ct.tag
# WHERE DATE(Timestamp) BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 50 DAY) AND CURRENT_DATE()
# """

# query_job = client.query(query)
# results = query_job.result()

# data = []   
# for row in results:     
#     data.append(dict(row))
    
# df = pd.DataFrame(data)

In [55]:
# df.head()

In [56]:
# df.to_parquet('140_days_df.parquet')

In [57]:
df = pd.read_parquet('140_days_df.parquet')

In [58]:
df_max_values = df.groupby(["descripcion", "Day", "Hour", "Minute"]).agg({"Value": "max"}).reset_index()

df_max_values['dayhourminute'] = df_max_values['Day'] + ' ' + df_max_values['Hour'] + ':' + df_max_values['Minute']
df_unpivot = df_max_values.pivot_table(index="dayhourminute", columns="descripcion", values="Value", aggfunc="max").reset_index()

In [59]:
df_unpivot

descripcion,dayhourminute,% BOMBA ALIM P45 M2 GLUCOSA,% BOMBA ALIMENT COLUMNAS 353509,% BOMBA ALIMENTACION CUBA M7,% BOMBA ALIMENTACION P45 M2 H0,% REG. VF BOMBA ALIMENT. SWENSON,% VAR. BOMBA SDA. CUBETA CH,% VARIADOR BOMBA ALIM. C.V.,% VF BOMBA ALIMENTACION W7,% VF BOMBA SALIDA SWENSON A CCHH,...,VOL RECUPERACION ANION GRUPO H0,VOL. RECUP. ANIÓN GRUPO 4,VOL. RECUP. CATIÓN GRUPO 4,VOLUMEN CIP A EDAR,VOLUMEN CIP A PEQUEÑAS AGUAS,VOLUMEN CIP LAVADO AGUA,VOLUMEN LAVADO CIP,VÁLVULA REGULADORA NIVEL M10,W10 STATUS,W7 STATUS
0,2024-06-09 00:00,0.000000,NaN,852.856995,9697.578125,NaN,NaN,51.768593,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-06-09 00:01,11.125526,NaN,1051.594116,10000.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-06-09 00:02,24.037436,NaN,1047.765991,9903.143555,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-06-09 00:03,27.494791,NaN,979.953308,10000.000000,60.657055,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-06-09 00:04,46.029263,NaN,899.699402,10000.000000,NaN,54.669937,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18212,2024-06-21 15:48,0.000000,NaN,NaN,NaN,NaN,14.933886,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18213,2024-06-21 15:49,NaN,NaN,NaN,0.000000,NaN,16.333483,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18214,2024-06-21 15:50,11.963161,NaN,NaN,4456.027344,NaN,15.153301,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18215,2024-06-21 15:51,20.796946,NaN,NaN,4819.892578,NaN,18.225300,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
columns_to_check = [
    "COR TITÁNIC AZÚCARES", 
    "COT TITÁNIC AZÚCARES NUEVO", 
    "COT AGUAS ÁCIDAS", 
    "COT AGUAS ÁCIDAS NUEVO"
]

conditions = pd.Series([False] * len(df_unpivot))
for col in columns_to_check:
    if col in df_unpivot.columns:
        conditions |= (df_unpivot[col] > 2500)

df_unpivot["flag"] = conditions.astype(int)

df_unpivot["dayhourminute"] = pd.to_datetime(df_unpivot["dayhourminute"])

df_unpivot.head()


descripcion,dayhourminute,% BOMBA ALIM P45 M2 GLUCOSA,% BOMBA ALIMENT COLUMNAS 353509,% BOMBA ALIMENTACION CUBA M7,% BOMBA ALIMENTACION P45 M2 H0,% REG. VF BOMBA ALIMENT. SWENSON,% VAR. BOMBA SDA. CUBETA CH,% VARIADOR BOMBA ALIM. C.V.,% VF BOMBA ALIMENTACION W7,% VF BOMBA SALIDA SWENSON A CCHH,...,VOL. RECUP. ANIÓN GRUPO 4,VOL. RECUP. CATIÓN GRUPO 4,VOLUMEN CIP A EDAR,VOLUMEN CIP A PEQUEÑAS AGUAS,VOLUMEN CIP LAVADO AGUA,VOLUMEN LAVADO CIP,VÁLVULA REGULADORA NIVEL M10,W10 STATUS,W7 STATUS,flag
0,2024-06-09 00:00:00,0.000000,NaN,852.856995,9697.578125,NaN,NaN,51.768593,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2024-06-09 00:01:00,11.125526,NaN,1051.594116,10000.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2024-06-09 00:02:00,24.037436,NaN,1047.765991,9903.143555,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,2024-06-09 00:03:00,27.494791,NaN,979.953308,10000.000000,60.657055,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,2024-06-09 00:04:00,46.029263,NaN,899.699402,10000.000000,NaN,54.669937,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [61]:
col_drop = ['COT AGUAS ÁCIDAS NUEVO', 'COT AGUAS ÁCIDAS', 'COR TITÁNIC AZÚCARES', 'COT TITÁNIC AZÚCARES NUEVO','dayhourminute']  
df = df_unpivot.drop(columns=[col for col in col_drop if col in df_unpivot.columns])

In [62]:
df = df.fillna(method='ffill').fillna(0)

C:\Users\JORGE\AppData\Local\Temp\ipykernel_17740\3594675012.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(0)


In [63]:
exclude_columns = ['flag']
df_corr = df.drop(columns=exclude_columns).corr().abs()

upper_triangle_mask = df_corr.where(np.triu(np.ones(df_corr.shape), k=1).astype(bool))

drop_columns = set()

for column in upper_triangle_mask.columns:
    high_corr = upper_triangle_mask[column][(upper_triangle_mask[column] > 0.8) & (upper_triangle_mask[column] < 1)]
    if not high_corr.empty:
        drop_columns.add(column)

df = df.drop(columns=list(drop_columns))

In [77]:
with open('dropped_columns.txt', 'w') as f:
    for item in drop_columns:
        f.write("%s\n" % item)

In [65]:
# one_hour_earlier = df_unpivot["dayhourminute"] - pd.Timedelta(hours=1)
# df_unpivot["flag"] |= df_unpivot["dayhourminute"].isin(one_hour_earlier[conditions])

# df_unpivot["flag"] = df_unpivot["flag"].astype(int)

# df_unpivot.head()

In [66]:
df

descripcion,% BOMBA ALIM P45 M2 GLUCOSA,% BOMBA ALIMENT COLUMNAS 353509,% BOMBA ALIMENTACION CUBA M7,% BOMBA ALIMENTACION P45 M2 H0,% REG. VF BOMBA ALIMENT. SWENSON,% VAR. BOMBA SDA. CUBETA CH,% VARIADOR BOMBA ALIM. C.V.,% VF BOMBA ALIMENTACION W7,% VF BOMBA SALIDA SWENSON A CCHH,BOMBA RECUPERACIÓN GRUPOS H0,...,VLV REG NIVEL CUBA SDA D.C.,VLV REG NIVEL FLASH SDA D.C.,VOL RECUP ANION GRUPOS GLUC,VOL. RECUP. ANIÓN GRUPO 4,VOL. RECUP. CATIÓN GRUPO 4,VOLUMEN CIP A PEQUEÑAS AGUAS,VOLUMEN CIP LAVADO AGUA,VOLUMEN LAVADO CIP,VÁLVULA REGULADORA NIVEL M10,flag
0,0.000000,0.0,852.856995,9697.578125,0.000000,0.000000,51.768593,0.0,0.000000,0.0,...,54.422516,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0
1,11.125526,0.0,1051.594116,10000.000000,0.000000,0.000000,51.768593,0.0,0.000000,0.0,...,53.573341,27.017218,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0
2,24.037436,0.0,1047.765991,9903.143555,0.000000,0.000000,51.768593,0.0,0.000000,0.0,...,51.552559,25.940605,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0
3,27.494791,0.0,979.953308,10000.000000,60.657055,0.000000,51.768593,0.0,0.000000,0.0,...,51.552559,27.621260,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0
4,46.029263,0.0,899.699402,10000.000000,60.657055,54.669937,51.768593,0.0,0.000000,0.0,...,45.102158,27.936863,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18212,0.000000,0.0,10000.000000,9115.012695,59.912788,14.933886,0.100000,0.0,59.556168,0.0,...,25.252186,21.817596,12.027593,0.0,0.0,0.0,0.0,0.0,100.0,0
18213,0.000000,0.0,10000.000000,0.000000,59.912788,16.333483,0.100000,0.0,59.556168,0.0,...,25.252186,21.817596,12.027593,0.0,0.0,0.0,0.0,0.0,100.0,0
18214,11.963161,0.0,10000.000000,4456.027344,59.912788,15.153301,0.100000,0.0,59.556168,0.0,...,25.252186,23.262724,12.027593,0.0,0.0,0.0,0.0,0.0,100.0,0
18215,20.796946,0.0,10000.000000,4819.892578,59.912788,18.225300,0.100000,0.0,59.556168,0.0,...,24.628326,20.543182,12.027593,0.0,0.0,0.0,0.0,0.0,100.0,0


In [67]:
X = df.drop('flag', axis=1)
X = X.fillna(0)
X.head()

descripcion,% BOMBA ALIM P45 M2 GLUCOSA,% BOMBA ALIMENT COLUMNAS 353509,% BOMBA ALIMENTACION CUBA M7,% BOMBA ALIMENTACION P45 M2 H0,% REG. VF BOMBA ALIMENT. SWENSON,% VAR. BOMBA SDA. CUBETA CH,% VARIADOR BOMBA ALIM. C.V.,% VF BOMBA ALIMENTACION W7,% VF BOMBA SALIDA SWENSON A CCHH,BOMBA RECUPERACIÓN GRUPOS H0,...,VLV REG NIVEL CUBA NODRIZA,VLV REG NIVEL CUBA SDA D.C.,VLV REG NIVEL FLASH SDA D.C.,VOL RECUP ANION GRUPOS GLUC,VOL. RECUP. ANIÓN GRUPO 4,VOL. RECUP. CATIÓN GRUPO 4,VOLUMEN CIP A PEQUEÑAS AGUAS,VOLUMEN CIP LAVADO AGUA,VOLUMEN LAVADO CIP,VÁLVULA REGULADORA NIVEL M10
0,0.000000,0.0,852.856995,9697.578125,0.000000,0.000000,51.768593,0.0,0.0,0.0,...,0.000000,54.422516,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11.125526,0.0,1051.594116,10000.000000,0.000000,0.000000,51.768593,0.0,0.0,0.0,...,0.000000,53.573341,27.017218,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,24.037436,0.0,1047.765991,9903.143555,0.000000,0.000000,51.768593,0.0,0.0,0.0,...,11.431318,51.552559,25.940605,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27.494791,0.0,979.953308,10000.000000,60.657055,0.000000,51.768593,0.0,0.0,0.0,...,10.341727,51.552559,27.621260,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,46.029263,0.0,899.699402,10000.000000,60.657055,54.669937,51.768593,0.0,0.0,0.0,...,10.341727,45.102158,27.936863,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
y = df['flag']
y.value_counts()

flag
0    15464
1     2753
Name: count, dtype: int64

In [69]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [70]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

param_grid = {
    'n_estimators': [200]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train_scaled, y_train)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Best Parameters: {grid_search.best_params_}')
print(f'Accuracy: {accuracy}')
print('Confusion Matrix:')
print(cm)
print('Classification Report:')
print(report)

Best Parameters: {'n_estimators': 200}
Accuracy: 0.9322173435784852
Confusion Matrix:
[[2962  140]
 [ 107  435]]
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.95      0.96      3102
           1       0.76      0.80      0.78       542

    accuracy                           0.93      3644
   macro avg       0.86      0.88      0.87      3644
weighted avg       0.93      0.93      0.93      3644



In [71]:
import joblib
model_filename = 'rf_model.pkl'
joblib.dump(best_model, model_filename)
print(f'Model saved as {model_filename}')

Model saved as rf_model.pkl


In [72]:
sclaer_filename = 'scaler_model.pkl'
joblib.dump(scaler, sclaer_filename)
print(f'Scaler saved as {sclaer_filename}')

Scaler saved as scaler_model.pkl


# CHECK

## COEFFICIENTS

In [73]:
feature_importances = best_model.feature_importances_
feature_names = X_train.columns if hasattr(X_train, 'columns') else [f'Feature {i}' for i in range(X_train_scaled.shape[1])]

top_20_indices = np.argsort(feature_importances)[-20:]

print("Top 20 feature importances:")
for idx in reversed(top_20_indices):
    print(f'Feature {feature_names[idx]}: {feature_importances[idx]}')

Top 20 feature importances:
Feature NIVEL NUEVO CISTERNA 19 GLUCOSA: 0.013702123695800188
Feature PRESION ENTRADA FILT GAF SDA W8: 0.013622915891127099
Feature NIVEL Nº1 TORTA P45 GLUCOSA: 0.013051159406097919
Feature NIVEL G4: 0.012493983803992834
Feature NIVEL CUBA ENZIMATICA Nº11: 0.011665175066369682
Feature NIVEL CUBA SOSA DILUIDA: 0.010130677369070024
Feature PRESION FILTRO NIAGRA Nº2 H0: 0.010019542222923849
Feature PRESION DIFERENCIAL FILT GAF W10: 0.009975690736026914
Feature NIVEL CUBA ENZIMATICA Nº13: 0.00989586962018817
Feature NIVEL 1 TORTA P45 H0: 0.009757181343660904
Feature NIVEL CRISTALIZADOR VERTICAL: 0.009667846392816992
Feature PRESION DIFERENCIAL FILT W-9: 0.00930646566493033
Feature NIVEL CISTERNA GLUCOSA Nº19: 0.009290966370337947
Feature PRESION DIF FILTRO TRANSP DX: 0.009109639797928098
Feature PRESION DIFERENCIAL FILT GAF W7: 0.008805091451455282
Feature NIVEL CUBA ENZIMATICA Nº17: 0.008576563286038834
Feature NIVEL CRISTALIZADOR HORIZONTAL 4: 0.00852273440984

## CORRECT OBSERVATIONS

In [74]:
X_scaled = scaler.transform(X)

df['predicted'] = best_model.predict(X_scaled) 

df['flag'] = df['predicted'] == df[y_test.name]
correct_predictions = df['flag'].sum()

print(f'Number of correctly predicted rows: {correct_predictions}')

total_predictions = len(df)
percentage_correct = (correct_predictions / total_predictions) * 100

print(f'Percentage of correctly predicted rows: {percentage_correct:.2f}%')

Number of correctly predicted rows: 17970
Percentage of correctly predicted rows: 98.64%


In [75]:
correct_predictions_0 = df[df['predicted'] == 0]['flag'].sum()
total_predictions_0 = df[df['predicted'] == 0].shape[0]
percentage_correct_0 = (correct_predictions_0 / total_predictions_0) * 100 if total_predictions_0 > 0 else 0

correct_predictions_1 = df[df['predicted'] == 1]['flag'].sum()
total_predictions_1 = df[df['predicted'] == 1].shape[0]
percentage_correct_1 = (correct_predictions_1 / total_predictions_1) * 100 if total_predictions_1 > 0 else 0

print(f'Percentage of correctly predicted 0s: {percentage_correct_0:.2f}%')
print(f'Percentage of correctly predicted 1s: {percentage_correct_1:.2f}%')

Percentage of correctly predicted 0s: 99.31%
Percentage of correctly predicted 1s: 94.97%


In [76]:
df

descripcion,% BOMBA ALIM P45 M2 GLUCOSA,% BOMBA ALIMENT COLUMNAS 353509,% BOMBA ALIMENTACION CUBA M7,% BOMBA ALIMENTACION P45 M2 H0,% REG. VF BOMBA ALIMENT. SWENSON,% VAR. BOMBA SDA. CUBETA CH,% VARIADOR BOMBA ALIM. C.V.,% VF BOMBA ALIMENTACION W7,% VF BOMBA SALIDA SWENSON A CCHH,BOMBA RECUPERACIÓN GRUPOS H0,...,VLV REG NIVEL FLASH SDA D.C.,VOL RECUP ANION GRUPOS GLUC,VOL. RECUP. ANIÓN GRUPO 4,VOL. RECUP. CATIÓN GRUPO 4,VOLUMEN CIP A PEQUEÑAS AGUAS,VOLUMEN CIP LAVADO AGUA,VOLUMEN LAVADO CIP,VÁLVULA REGULADORA NIVEL M10,flag,predicted
0,0.000000,0.0,852.856995,9697.578125,0.000000,0.000000,51.768593,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,True,0
1,11.125526,0.0,1051.594116,10000.000000,0.000000,0.000000,51.768593,0.0,0.000000,0.0,...,27.017218,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,True,0
2,24.037436,0.0,1047.765991,9903.143555,0.000000,0.000000,51.768593,0.0,0.000000,0.0,...,25.940605,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,True,0
3,27.494791,0.0,979.953308,10000.000000,60.657055,0.000000,51.768593,0.0,0.000000,0.0,...,27.621260,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,True,0
4,46.029263,0.0,899.699402,10000.000000,60.657055,54.669937,51.768593,0.0,0.000000,0.0,...,27.936863,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18212,0.000000,0.0,10000.000000,9115.012695,59.912788,14.933886,0.100000,0.0,59.556168,0.0,...,21.817596,12.027593,0.0,0.0,0.0,0.0,0.0,100.0,True,0
18213,0.000000,0.0,10000.000000,0.000000,59.912788,16.333483,0.100000,0.0,59.556168,0.0,...,21.817596,12.027593,0.0,0.0,0.0,0.0,0.0,100.0,True,0
18214,11.963161,0.0,10000.000000,4456.027344,59.912788,15.153301,0.100000,0.0,59.556168,0.0,...,23.262724,12.027593,0.0,0.0,0.0,0.0,0.0,100.0,True,0
18215,20.796946,0.0,10000.000000,4819.892578,59.912788,18.225300,0.100000,0.0,59.556168,0.0,...,20.543182,12.027593,0.0,0.0,0.0,0.0,0.0,100.0,True,0
